In [1]:
from dms_stan.datasets.trpb import load_trpb_dataset, TrpBGrowthModel

Prior predictive check for the TrpB datasets.

In [2]:
data = load_trpb_dataset(
    "~/GitRepos/DMSStan/raw_data/trpb/3-site_merged_replicates/LibI/20230926/LibI_merged_AAs.csv"
)
model = TrpBGrowthModel(**data)
model.prior_predictive()

BokehModel(combine_events=True, render_bundle={'docs_json': {'7c30acb3-dacb-4f23-b6f9-5ad2117572bb': {'version…

In [3]:
model.draw(5, as_xarray=True)

<xarray.Dataset> Size: 9MB
Dimensions:                  (n: 5, a: 9121, c: 2, b: 5)
Coordinates:
    theta_tg0.dist1.dist1.t  (b) float64 40B 0.3333 0.4444 0.5556 0.6667 1.0
Dimensions without coordinates: n, a, c, b
Data variables:
    log_A                    (n, a) float64 365kB 0.2028 0.6937 ... 0.3252
    theta_t0                 (n, a) float64 365kB 7.174e-05 ... 8.146e-05
    starting_counts          (n, a) int64 365kB 258 372 734 382 ... 190 351 298
    r_mean                   (n, a) float64 365kB 0.471 0.09309 ... 0.02502
    r_std                    (n) float64 40B 0.2142 0.1911 0.2052 0.4672 0.4793
    r                        (n, c, a) float64 730kB 0.4085 0.2426 ... -1.262
    theta_tg0                (n, c, b, a) float64 4MB 7.933e-05 ... 1.845e-05
    timepoint_counts         (n, c, b, a) int64 4MB 342 592 950 ... 145 881 95

# MAP

Now that we've selected our priors, we're ready to identify the MAP:

In [ ]:
map_ = model.approximate_map()
map_.plot_loss_curve()

Epochs:   0%|          | 80/100000 [00:01<18:37, 89.43it/s, -log pdf/pmf=61879575.37]  

# MCMC

We can now use Stan to sample from the posterior:

In [ ]:
# samples = model.mcmc(
#     output_dir="./tmp",
#     cpp_options={"STAN_THREADS": True},
# )